In [15]:
import json
import logging
import random
import os
import pandas as pd
from langchain_core.prompts import ChatPromptTemplate

In [3]:
# Load relation definitions (adjust the file path as needed)
with open("inputs/relation_types.json", "r") as f:
    relation_def = json.load(f)

# Create a mapping from relation type to id
relation_2_id = {rel: idx for idx, rel in enumerate(list(relation_def.keys()))}

# Load candidate triples from your candidate file (output from step 2)
candidate_triples = []
input_file = "outputs/sample/triples_sample.json"  # adjust as necessary
with open(input_file, "r") as f:
    for line in f:
        t = json.loads(line)
        candidate_triples.append((t['s'], t['p'], t['o']))

print(f"Loaded {len(candidate_triples)} candidate triples")

Loaded 281 candidate triples


In [4]:
candidate_triples

[('natural language explanation', 'Evaluate-for', 'model reasoning'),
 ('natural language explanation',
  'Evaluate-for',
  'classification performance'),
 ('natural language explanation', 'Used-for', 'improving classification'),
 ('natural language explanation', 'Used-for', 'justifying predictions'),
 ('natural language explanation', 'Evaluate-for', 'inconsistencies detection'),
 ('natural language explanation', 'Evaluate-for', 'gender-fairness'),
 ('natural language explanation', 'Evaluate-for', 'ambiguity resolution'),
 ('natural language explanation', 'Part-of', 'natural language processing'),
 ('natural language explanation', 'Hyponym-Of', 'text explanation'),
 ('natural language explanation', 'Compare', 'text-to-image disambiguation'),
 ('natural language explanation',
  'Used-for',
  'enhancing language representation'),
 ('natural language explanation', 'Used-for', 'guiding model reasoning'),
 ('event relation', 'Used-for', 'event detection'),
 ('event detection', 'Used-for', '

In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
# load sample data
data = json.load(open('I:/11_DFKI_Hiwi/Work/01_Code/Graphusion/outputs/sample/concept_abstracts_sample.json', 'r'))
relation_types = json.load(open('I:/11_DFKI_Hiwi/Work/01_Code/Graphusion/inputs/relation_types.json', 'r'))

In [7]:
# Initialize your model (ensure OPENAI_API_KEY is set)
from src.models import KnowledgeGraphLLM
model = KnowledgeGraphLLM(model_name="gpt-4o", max_tokens=10000, api_key=os.getenv("OPENAI_API_KEY"))

# Configuration for the fusion process
config = {
    "max_input_char": 10000,  # Maximum characters from abstracts to include
    # You could add other keys like "refined_concepts_file" or "annotated_graph_file" if needed.
}

In [8]:
## fused triples path
fused_triples_path = "outputs/sample/original_prompt/fused_triples_sample.json"

In [ ]:
from tqdm import tqdm

def fuse_triples(model, candidate_triples, relation_def, data):
    
    fused_triples = []
    # For each unique concept (from candidate triples), create a textual summary of its related triples
    candidate_concepts = set(data.keys())

    for concept in tqdm(list(candidate_concepts)):
        # Create a simple string representation (list all triples involving this concept)
        candidate_subgraph = []
        for triple in candidate_triples:
            if concept in triple:
                candidate_subgraph.append(f"({triple[0]}, {triple[1]}, {triple[2]})")
                
        # Get background abstracts (if available)
        background = ""
        if concept in data:
            background = ' --  '.join(data[concept]['abstracts'])
        else:
            print(f"No background found for concept {concept}")
            background = "No background information available."
        
        # Build the fusion prompt using a simple formatted string
        prompt_template_txt = open("prompts/prompt_fusion.txt").read()
        # print(prompt_template_txt)
        prompt_template = ChatPromptTemplate.from_template(prompt_template_txt)
        # print('Prompt template', prompt_template)   
        
        prompt = prompt_template.invoke({
            "concept": concept,
            "graph1": candidate_subgraph,
            "graph2": fused_triples,
            "background": background,
            "relation_definitions": '\n'.join([f"{k}: {v['description']}" for k, v in relation_def.items()])
        })

        response = model.invoke(prompt)
        print(f"Response for concept {concept}: {response}")    

        print("/n /n /n") 

        if response and response != "None":
            try:
                response_json = json.loads(response)
                for triple in response_json:
                    # Validate the relation exists in your definitions
                    if triple['p'] in relation_def:
                        fused_triples.append((triple['s'], triple['p'], triple['o']))
            except Exception as e:
                print(f"Error fusing triples for concept {concept}: {e}")
        
        fused_triples = list(set(fused_triples))
        print(f"Total fused triples: {len(fused_triples)}")
        
    return fused_triples


In [ ]:
fused_triples = fuse_triples(model, candidate_triples, relation_def, data)

  5%|▌         | 1/20 [00:06<01:59,  6.28s/it]

Response for concept text rationale: [{"s": "text rationale", "p": "Used-for", "o": "explainable NLP"}, {"s": "text rationale", "p": "Evaluate-for", "o": "factual consistency"}, {"s": "text rationale", "p": "Evaluate-for", "o": "rationale-label pairs"}, {"s": "text rationale", "p": "Evaluate-for", "o": "new information"}, {"s": "text rationale", "p": "Evaluate-for", "o": "human utility"}, {"s": "text rationale", "p": "Evaluate-for", "o": "reasoning and prediction processes"}, {"s": "text rationale", "p": "Evaluate-for", "o": "free-text rationales"}, {"s": "text rationale", "p": "Evaluate-for", "o": "chain-of-thought"}, {"s": "text rationale", "p": "Evaluate-for", "o": "model's reasoning"}]
/n /n /n
Total fused triples: 9


 10%|█         | 2/20 [00:13<02:02,  6.78s/it]

Response for concept relation extraction: [{"s": "relation extraction", "p": "Part-of", "o": "information extraction"}, {"s": "pre-trained language models", "p": "Used-for", "o": "relation extraction"}, {"s": "document-level relation extraction", "p": "Part-of", "o": "relation extraction"}, {"s": "relation extraction", "p": "Used-for", "o": "acquisition of relational facts"}, {"s": "text rationale", "p": "Evaluate-for", "o": "rationale-label pairs"}, {"s": "text rationale", "p": "Used-for", "o": "explainable NLP"}, {"s": "text rationale", "p": "Evaluate-for", "o": "chain-of-thought"}, {"s": "text rationale", "p": "Evaluate-for", "o": "new information"}, {"s": "text rationale", "p": "Evaluate-for", "o": "reasoning and prediction processes"}, {"s": "text rationale", "p": "Evaluate-for", "o": "free-text rationales"}, {"s": "text rationale", "p": "Evaluate-for", "o": "factual consistency"}, {"s": "text rationale", "p": "Evaluate-for", "o": "human utility"}, {"s": "text rationale", "p": "Ev

 15%|█▌        | 3/20 [00:22<02:11,  7.72s/it]

...

Response for concept summarization dataset: [{"s": "summarization dataset", "p": "Part-of", "o": "dialogue summarization"}, {"s": "summarization dataset", "p": "Part-of", "o": "cross-lingual summarization"}, {"s": "summarization dataset", "p": "Part-of", "o": "multi-document summarization"}, {"s": "summarization dataset", "p": "Part-of", "o": "extractive summarization"}, {"s": "summarization dataset", "p": "Part-of", "o": "abstractive summarization"}, {"s": "summarization dataset", "p": "Part-of", "o": "human evaluation"}, {"s": "summarization dataset", "p": "Part-of", "o": "few-shot summarization"}, {"s": "summarization dataset", "p": "Evaluate-for", "o": "summarization performance"}, {"s": "summarization dataset", "p": "Used-for", "o": "pre-training models"}, {"s": "summarization dataset", "p": "Used-for", "o": "fine-tuning models"}, {"s": "summarization dataset", "p": "Used-for", "o": "evaluation metrics development"}, {"s": "summarization dataset", "p": "Used-for", "o": "benchmarki

 95%|█████████▌| 19/20 [12:53<01:19, 79.57s/it]

Response for concept summarization model: [{"s": "RSTformer", "p": "Hyponym-Of", "o": "summarization model"}, {"s": "summarization dataset", "p": "Evaluate-for", "o": "summarization performance"}, {"s": "question answering", "p": "Part-of", "o": "semantic parsing"}, {"s": "natural language explanation", "p": "Hyponym-Of", "o": "text explanation"}, {"s": "document-level relation extraction", "p": "Part-of", "o": "relation extraction"}, {"s": "question answering qa", "p": "Part-of", "o": "multi-document downstream tasks"}, {"s": "reasoning datasets", "p": "Part-of", "o": "question answering"}, {"s": "summarization evaluation", "p": "Evaluate-for", "o": "Abstractive summarization"}, {"s": "form question answering", "p": "Conjunction", "o": "instruction following"}, {"s": "text generation", "p": "Used-for", "o": "generating free-text rationales"}, {"s": "CrossSum", "p": "Part-of", "o": "summarization system"}, {"s": "text rationale", "p": "Evaluate-for", "o": "model's reasoning"}, {"s": "r

100%|██████████| 20/20 [14:23<00:00, 43.16s/it]

Response for concept question answering: [{"s": "question answering", "p": "Used-for", "o": "factual consistency evaluation"}, {"s": "question answering", "p": "Part-of", "o": "open-domain question answering"}, {"s": "question answering", "p": "Used-for", "o": "generating useful intermediate context"}, {"s": "question answering", "p": "Used-for", "o": "retrieval model evaluation"}, {"s": "question answering", "p": "Used-for", "o": "elaboration generation"}, {"s": "question answering", "p": "Used-for", "o": "commonsense reasoning"}, {"s": "question answering", "p": "Used-for", "o": "cross-document informational relations"}, {"s": "question answering", "p": "Used-for", "o": "semantic representation"}, {"s": "question answering", "p": "Used-for", "o": "Table Question Answering (TQA)"}, {"s": "question answering", "p": "Used-for", "o": "temporal reasoning"}, {"s": "question answering", "p": "Used-for", "o": "information retrieval"}, {"s": "question answering", "p": "Used-for", "o": "long-f

In [ ]:
len(fused_triples)

183

In [ ]:
# saving triples list to a text file

with open(fused_triples_path, "w") as f:
    for t in fused_triples:
        f.write(json.dumps({"s": t[0], "p": t[1], "o": t[2]}) + "\n")

In [ ]:
from neo4j import GraphDatabase

class Neo4jTripleStore:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
    
    def close(self):
        self.driver.close()
    
    def insert_triple(self, subject, relation, obj):
        with self.driver.session() as session:
            session.execute_write(self._create_triple, subject, relation, obj)
    
    @staticmethod
    def _create_triple(tx, subject, relation, obj):
        query = """
        MERGE (s:Concept {name: $subject})
        MERGE (o:Concept {name: $object})
        MERGE (s)-[r:RELATION {type: $relation}]->(o)
        """
        tx.run(query, subject=subject, relation=relation, object=obj)


In [ ]:
neo4j_store = Neo4jTripleStore(uri="neo4j+s://68cfeea5.databases.neo4j.io", user="neo4j", password="Z2D8tI5LWCCkcxR7Fbp0nb5X4V0sxwhiHwxOtXHe-Dc")
for s, p, o in fused_triples:
    neo4j_store.insert_triple(s, p, o)
neo4j_store.close()
print("Stored all fused triples into Neo4j.")


Stored all fused triples into Neo4j.


In [ ]:
## fused triples path
fused_triples_path = "outputs/sample/modified_prompt/fused_triples_sample.json"